In [15]:
import sys
sys.path.insert(0, "/Users/rodrigo/google_drive/code/")

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import votable, ascii
import json
import numpy as np
from types import SimpleNamespace
from astropy.table import Table, vstack
import ps_api

### SQL query on MAST

In [22]:
def z_cat_query(tl, br):
    string = "select * from catalogRecordRowStore where raMean>{:.2f} and raMean<{:.2f} and decMean>{:.2f} and decMean<{:.2f}".format(br[0], tl[0], br[1], tl[1])
    return string

In [23]:
def sexa_to_deci(c):
    sc = SkyCoord(c, unit=(u.hourangle, u.deg))
    return sc.ra.deg, sc.dec.deg

In [28]:
tl = sexa_to_deci("23:21:48.8754345 +29:21:46.953459")
br = sexa_to_deci("23:20:25.6116039 +29:04:31.197087")
print(z_cat_query(tl, br))

select * from catalogRecordRowStore where raMean>350.11 and raMean<350.45 and decMean>29.08 and decMean<29.36


### Preparing prioritize.py input

Mask A

In [8]:
center = "23:21:08.3381708 +29:13:00.035871"
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
print('hour degree')
print(sc.ra.hour, sexa_to_deci(center)[1])
print('\ndegree degree')
print(*sexa_to_deci(center))

hour degree
23.35231615855556 29.21667663083333

degree degree
350.2847423783333 29.21667663083333


Mask B

In [16]:
center = "00:03:50.0443707 +10:04:22.110113"
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
print('hour deg:')
print(sc.ra.hour, sexa_to_deci(center)[1])
print('\ndeg deg:')
print(*sexa_to_deci(center))

hour deg:
0.06390121408333334 10.072808364722222

deg deg:
0.9585182112499999 10.072808364722222


### Downloading the PS catalog

In [29]:
# cl_pos = ps_api.resolve('RMJ000343.8+100123.8')
cl_pos = sexa_to_deci(center)
cl_pos

(350.2847423783333, 29.21667663083333)

In [30]:
# radius = (9 * u.arcmin).to(u.deg).value
radius = (12 * u.arcmin).to(u.deg).value
# radius = .03

In [31]:
# constraints = {'nDetections.gt':1, 'rMeanKronMag.gt':-990}
# constraints = {'rMeanKronMag.gt':-990}
# constraints = {'nDetections.gt':10}
constraints = {'nDetections.gt':1}
# constraints = {}

In [32]:
columns = """objID,raMean,decMean,rMeanKronMag,nDetections""".split(',')
# strip blanks and weed out blank and commented-out values
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

In [33]:
results = ps_api.ps1cone(*cl_pos, radius, columns=columns, **constraints)
# try including release="dr2" as an argument

In [34]:
with open('ps_api_res_nDetections1.csv', 'w') as f:
    f.write(results)

### Converting archival redshift file into csv

In [9]:
archival = ascii.read('archivalz.txt', names = ['RA', 'DEC', 'z', 'delta_z'])
archival.write('archivalz.csv', format='ascii.csv')

### Converting output.txt into catalog

In [22]:
io_dir = '1/dsim/'
# io_dir = '2/dsim/'
columns = ['OBJNAME', 'RA', 'DEC', 'EQX', 'MAG', 'band', 'PCODE', 'LIST', 'SEL?', 'PA', 'L1', 'L2']
dtype = []

for column in columns:
    if column=='SEL?':
        dtype.append('i4')
    else:
        dtype.append('<U12')

def read_target_file(filename):
    with open(filename,'r') as targets:
        lines = targets.readlines()

    align = []
    targets = []
    for line in lines:
        line_data = line.strip().split()
        if len(line_data)==9:
            align.append(line_data)
        elif len(line_data)==10:
            targets.append(line_data)
    return align, targets


In [23]:
infilename = "{}output.txt".format(io_dir)
out_selected = "{}selected.csv".format(io_dir)
out_not_selected = "{}not_selected.csv".format(io_dir)
out_align = "{}align.csv".format(io_dir)

In [24]:
align, targets = read_target_file(infilename)

In [25]:
df_targets = Table(np.array(targets), names=columns[:-2], dtype=dtype[:-2])
df_align = Table(np.array(align[:-1]), names=columns[:-3], dtype=dtype[:-3])

In [26]:
df_targets[df_targets['SEL?']==1].write(out_selected, format='ascii.csv', overwrite=True)
df_targets[df_targets['SEL?']==0].write(out_not_selected, format='ascii.csv', overwrite=True)
df_align.write(out_align, format='ascii.csv', overwrite=True)

In [27]:
df_targets[df_targets['SEL?']==1]

OBJNAME,RA,DEC,EQX,MAG,band,PCODE,LIST,SEL?,PA
str12,str12,str12,str12,str12,str12,str12,str12,int32,str12
9045648334,00:03:37.158,+09:57:23.64,2000.0,21.00,r,637,1,1,-98
8541609913,00:03:25.014,+09:58:58.10,2000.0,20.61,r,4961,1,1,-98
9363621614,00:03:44.726,+09:58:33.22,2000.0,20.88,r,744,1,1,-98
9340009833,00:03:44.164,+09:58:27.83,2000.0,20.48,r,4232,1,1,-98
8937131009,00:03:34.515,+10:00:01.01,2000.0,20.73,r,1970,1,1,-98
9097560626,00:03:38.352,+09:59:30.18,2000.0,19.75,r,14642,1,1,-98
8880001157,00:03:33.128,+10:01:31.96,2000.0,20.63,r,2364,1,1,-98
9190586401,00:03:40.579,+10:01:47.58,2000.0,20.66,r,3906,1,1,-98
9202491693,00:03:40.861,+10:01:33.55,2000.0,20.07,r,5845,1,1,-98
